In [7]:
import numpy as np
import skimage
from skimage import io
from matplotlib import pyplot as plt
from skimage import data
from skimage.transform import resize
from skimage.metrics import peak_signal_noise_ratio, structural_similarity

In [8]:
#Gaussian Filter

def gaussian_kernel(k_size, sigma):
    size = k_size//2
    y, x = np.ogrid[-size:size+1, -size:size+1]
    filter = 1/(2*np.pi * (sigma**2)) * np.exp(-1 *(x**2 + y**2)/(2*(sigma**2)))
    sum = filter.sum()
    filter /= sum
    return filter

def padding(img, k_size):
    pad_size = k_size//2
    h, w, ch = img.shape
    
    res = np.zeros((h + (2*pad_size), w+(2*pad_size), ch), dtype=np.float)
    
    if pad_size == 0:
        res = img.copy()
    else:
        res[pad_size:-pad_size, pad_size:-pad_size] = img.copy()
    return res

def gaussian_filtering(img, k_size=5,sigma=4):
    h, w, ch = img.shape
    filter = gaussian_kernel(k_size, sigma)
    pad_img = padding(img,k_size)
    filtered_img = np.zeros((h, w, ch), dtype=np.float32)
    
    for ch in range(0, ch):
        for i in range(h):
            for j in range(w):
                filtered_img[i, j, ch] = np.sum(filter * pad_img[i:i+k_size, j:j+k_size, ch])

    return filtered_img

In [9]:
#median filter

def median_filter(img, filter_size=(3, 3), stride=1):
    img_shape = np.shape(img)
    result_shape = tuple(np.int64((np.array(img_shape[:2]) - np.array(filter_size)) / stride) + 1) + (img_shape[2],)
    result = np.zeros(result_shape)

    for h in range(0, result_shape[0], stride):
        for w in range(0, result_shape[1], stride):
            for c in range(img_shape[2]):
                tmp = img[h:h + filter_size[0], w:w + filter_size[1], c].ravel()
                tmp = np.sort(tmp)
                result[h, w, c] = tmp[int(len(tmp) / 2)]

    return result

In [10]:
def fast_bilateral_filter(noisy_img, k_size=5, sigma_space=4, sigma_intensity=0.2):
    h, w, ch = noisy_img.shape
    bilateral_noisy_img = np.zeros((h, w, ch))

    spatial_filter = gaussian_kernel(k_size, sigma_space)

    for c in range(ch):
        intensity_center = noisy_img[:, :, c]
        weighted_sum = np.zeros_like(intensity_center)
        normalization_factor = np.zeros_like(intensity_center)

        for m in range(-k_size//2, k_size//2 + 1):
            for n in range(-k_size//2, k_size//2 + 1):
                i_neighbors = np.clip(np.arange(h) + m, 0, h - 1)
                j_neighbors = np.clip(np.arange(w) + n, 0, w - 1)
                intensity_neighbors = noisy_img[i_neighbors, :, c][:, j_neighbors]
                weight_intensity = np.exp(-(intensity_center - intensity_neighbors)**2 / (2 * sigma_intensity**2))
                weight_spatial = spatial_filter[m + k_size//2, n + k_size//2]
                weighted_sum += intensity_neighbors * weight_intensity * weight_spatial
                normalization_factor += weight_intensity * weight_spatial

        bilateral_noisy_img[:, :, c] = weighted_sum / normalization_factor
    return bilateral_noisy_img

In [11]:
def non_local_mean_optimized(image, h=0.1, search_window=5, patch_size=3):
    padded_img = np.pad(image, ((search_window, search_window), (search_window, search_window), (0, 0)), mode='symmetric')

    result = np.zeros_like(image, dtype=np.float64)

    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            patch_i = slice(i, i + 2 * search_window + 1)
            patch_j = slice(j, j + 2 * search_window + 1)

            patch = padded_img[patch_i, patch_j, :]
            target = padded_img[i:i + 2 * search_window + 1, j:j + 2 * search_window + 1, :]

            distances = np.sum((patch - target) ** 2, axis=(0, 1, 2))
            weights = np.exp(-distances / (2 * h ** 2))
            normalized_weights = weights / np.sum(weights)

            result[i, j, :] = np.sum(target * normalized_weights, axis=(0, 1))
	
    return (result - np.min(result)) / (np.max(result) - np.min(result))

In [12]:
names = ['baby', 'bagles', 'beach', 'book', 'dog', 'girl_ani', 'lego', 'kitty', 'house', 'street']

for file_name in names:
    noisy_img = io.imread("/Users/jiyoung/Desktop/univ/'23-2/Engineering Design/workspace/" + file_name + "_noisy.png")
    noisy_img = noisy_img[:,:,0:3]
    noisy_img = noisy_img/255.0
    clean_img = io.imread("/Users/jiyoung/Desktop/univ/'23-2/Engineering Design/workspace/" + file_name + ".png")
    clean_img = clean_img[:,:,0:3]
    clean_img = clean_img/255.0
    
    title = "Denoising Image: " + file_name
    print(title + "\n")
    
    filtered_median = median_filter(noisy_img)
    median_img_resized = resize(filtered_median, clean_img.shape, mode ='constant', anti_aliasing = True)
    median_img_resized = np.clip(median_img_resized, 0., 1.0)
    filtered_bilateral = fast_bilateral_filter(median_img_resized, k_size=5, sigma_space=4, sigma_intensity=0.2)
    filtered_NLMeans = non_local_mean_optimized(filtered_bilateral, h=0.1, search_window=5, patch_size=3)

    PSNR = peak_signal_noise_ratio(clean_img, filtered_NLMeans)
    SSIM = structural_similarity(clean_img, filtered_NLMeans, multichannel=True)

    print("PSNR : " + str(PSNR)); print("SSIM : " + str(SSIM)); print("\n")

Denoising Image: baby



/var/folders/n1/t3hm6b612gg69mlfjrdv4ht00000gn/T/ipykernel_40942/3832692407.py:21: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  SSIM = structural_similarity(clean_img, filtered_NLMeans, multichannel=True)


PSNR : 27.871877661839058
SSIM : 0.8646022385330245


Denoising Image: bagles

PSNR : 22.851888395658555
SSIM : 0.7515110582879334


Denoising Image: beach

PSNR : 25.505322916003088
SSIM : 0.8678247210359232


Denoising Image: book

PSNR : 25.826063064902016
SSIM : 0.881804775133794


Denoising Image: dog

PSNR : 26.800016061810283
SSIM : 0.7921121344933643


Denoising Image: girl_ani

PSNR : 23.473612409702906
SSIM : 0.8670063584030204


Denoising Image: lego

PSNR : 22.72685545225771
SSIM : 0.7709218782640028


Denoising Image: kitty

PSNR : 27.337721404933696
SSIM : 0.8826653049718028


Denoising Image: house

PSNR : 19.831177597004036
SSIM : 0.568343645702536


Denoising Image: street

PSNR : 21.170133291788837
SSIM : 0.6238138896388657


